In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
from NSPLIB.src.instances.tanksize.tanksize import const_model
tanksize_obj = 0.9030394623070541
tanksize_sol = {"productTankSize[1]": 659.2053849850757, "productTankSize[2]": 552.3954188143167, "productTankSize[3]": 225.73997707963284}
tanksize_y_bound = {"productTankSize[1]": [643, 4018.36], "productTankSize[2]": [536, 3348], "productTankSize[3]": [214, 1339.45]}
tanksize_y_bound = {"productTankSize[1]": [643, 670], "productTankSize[2]": [536, 560], "productTankSize[3]": [214, 230]}
sto_m = const_model()
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()

v_f= ValueFunction(m,solver="baron")
y_dimension = {'productTankSize[1]':{}, 'productTankSize[2]':{}, 'productTankSize[3]':{}}
steps=10
for y in tanksize_y_bound:
    v_f.calc_1D(idx=y, y_val_fix=tanksize_sol,
                    interval= tanksize_y_bound[y], step = steps)
    for s in m.scenarios:
        y_dimension[y][s] = v_f.value_func[s]
    y_dimension[y]['total'] = v_f.total_value_func


# Specify grid dimensions
n_rows = len(y_dimension)
n_cols = len(m.scenarios) + 1  # +1 for 'total'

# Create a grid of Output widgets for the plots
outputs = [[widgets.Output() for _ in range(n_cols)] for _ in range(n_rows)]

# Fill each plot with the corresponding value function data
for i, y_key in enumerate(y_dimension.keys()):
    for j in range(n_cols):
        with outputs[i][j]:
            x = np.linspace(tanksize_y_bound[y_key][0], tanksize_y_bound[y_key][1], len(y_dimension[y_key]['total']))
            if j < len(m.scenarios):
                scenario_key = list(m.scenarios)[j]
                y_vals = y_dimension[y_key][scenario_key]
                title = f"{y_key} - Scenario {scenario_key}"
            else:
                y_vals = y_dimension[y_key]['total']
                title = f"{y_key} - Total"
            fig, ax = plt.subplots(figsize=(4,4))
            ax.plot(x, y_vals, marker='o')
            ax.set_title(title)
            ax.set_xlabel(y_key)
            ax.set_ylabel("Value")
            plt.tight_layout()
            plt.show()

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(n_rows) for j in range(n_cols)],
    layout=widgets.Layout(grid_template_columns=f"repeat({n_cols}, 400px)")
)

display(grid)

GridBox(children=(Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Ou…